In [ ]:
import os
import pandas as pd

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/reconstruction_data2.xlsx')
data.head()

,id,음식,연락처,성별,연령,구,연도,월,일,날씨,...,반반피자,불고기피자,양념치킨,오리지널피자,짜장면,짬뽕,콤비네이션피자,후라이드치킨,메뉴2,만족도
0,1,중식,105160345,남,50대,연제구,2022,1,11,맑음,...,0,0,0,0,0,1,0,0,짬뽕,0
1,2,중식,104901583,여,60대 이상,연제구,2022,1,13,맑음,...,0,0,0,0,0,1,0,0,짬뽕,1
2,3,중식,108304745,여,60대 이상,연제구,2022,1,13,맑음,...,0,0,0,0,0,1,0,0,짬뽕,4
3,4,중식,107651082,여,40대,동래구,2022,1,20,맑음,...,0,0,0,0,1,0,0,0,짜장면,1
4,5,중식,107984063,남,30대,동래구,2022,1,21,맑음,...,0,1,0,0,0,0,0,0,불고기피자,2


In [276]:
raw_data=data[['성별','연령','월','날씨','최저기온','최고기온','메뉴2']]
raw_data

,성별,연령,월,날씨,최저기온,최고기온,메뉴2
0,남,50대,1,맑음,-3.3,6.7,짬뽕
1,여,60대 이상,1,맑음,-3.8,2.8,짬뽕
2,여,60대 이상,1,맑음,-3.8,2.8,짬뽕
3,여,40대,1,맑음,-3.3,8.1,짜장면
4,남,30대,1,맑음,-1.0,9.8,불고기피자
...,...,...,...,...,...,...,...
11650,남,30대,8,맑음,27.3,33.4,반반치킨
11651,여,40대,8,맑음,28.1,34.7,불고기피자
11652,남,30대,8,맑음,23.5,31.0,반반치킨
11653,여,20대,8,맑음,24.6,32.1,양념치킨


In [277]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [278]:
raw_data_copy = raw_data.copy()
raw_data_copy.loc[:,'profile'] = raw_data[['성별', '연령', '월', '날씨', '최저기온', '최고기온']].astype(str).agg(' '.join, axis=1)
raw_data_copy

,성별,연령,월,날씨,최저기온,최고기온,메뉴2,profile
0,남,50대,1,맑음,-3.3,6.7,짬뽕,남 50대 1 맑음 -3.3 6.7
1,여,60대 이상,1,맑음,-3.8,2.8,짬뽕,여 60대 이상 1 맑음 -3.8 2.8
2,여,60대 이상,1,맑음,-3.8,2.8,짬뽕,여 60대 이상 1 맑음 -3.8 2.8
3,여,40대,1,맑음,-3.3,8.1,짜장면,여 40대 1 맑음 -3.3 8.1
4,남,30대,1,맑음,-1.0,9.8,불고기피자,남 30대 1 맑음 -1.0 9.8
...,...,...,...,...,...,...,...,...
11650,남,30대,8,맑음,27.3,33.4,반반치킨,남 30대 8 맑음 27.3 33.4
11651,여,40대,8,맑음,28.1,34.7,불고기피자,여 40대 8 맑음 28.1 34.7
11652,남,30대,8,맑음,23.5,31.0,반반치킨,남 30대 8 맑음 23.5 31.0
11653,여,20대,8,맑음,24.6,32.1,양념치킨,여 20대 8 맑음 24.6 32.1


In [279]:
# TF-IDF 변환
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(raw_data_copy['profile'])

In [280]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [284]:
def recommend_menu(user_index, num_recommendations=3):
    # 주어진 사용자와 다른 사용자들 간의 유사도 점수
    sim_scores = list(enumerate(cosine_sim[user_index]))

    # 유사도에 따라 사용자들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 추천 메뉴 리스트 초기화
    recommended_menus = []

    # 추천 메뉴 선택
    for i in sim_scores:
        if len(recommended_menus) < num_recommendations:
            menu = raw_data_copy['메뉴2'].iloc[i[0]]
            if menu not in recommended_menus:  # 중복 방지
                recommended_menus.append(menu)
        else:
            break

    return recommended_menus

In [332]:
import random

# 데이터셋에서 무작위로 10개의 사용자 인덱스 선택
random_user_indices = random.sample(range(len(raw_data_copy)), 10)

# 결과를 저장할 빈 리스트
results = []

# 각 사용자에 대한 추천 메뉴 수행 및 해당 정보 추출
for user_index in random_user_indices:
    # 사용자 정보 추출
    user_info = raw_data_copy.loc[user_index, ['연령', '월', '날씨', '최저기온', '최고기온']]

    # 실제 메뉴와 추천 메뉴
    actual_menu = raw_data_copy['메뉴2'][user_index]
    recommended_menus = recommend_menu(user_index)

    # 결과 데이터 프레임에 추가
    results.append({
        '사용자 인덱스': user_index,
        '연령': user_info['연령'],
        '월': user_info['월'],
        '날씨': user_info['날씨'],
        '최저기온': user_info['최저기온'],
        '최고기온': user_info['최고기온'],
        '실제 정답': actual_menu,
        '추천메뉴 1': recommended_menus[0] if len(recommended_menus) > 0 else 'N/A',
        '추천메뉴 2': recommended_menus[1] if len(recommended_menus) > 1 else 'N/A',
        '추천메뉴 3': recommended_menus[2] if len(recommended_menus) > 2 else 'N/A'
    })

# 결과를 데이터 프레임으로 변환
result_df = pd.DataFrame(results)

# 결과 출력
result_df

,사용자 인덱스,연령,월,날씨,최저기온,최고기온,실제 정답,추천메뉴 1,추천메뉴 2,추천메뉴 3
0,4550,40대,11,맑음,7.5,18.6,불고기피자,불고기피자,짜장면,오리지널피자
1,2962,50대,11,맑음,8.9,18.6,짬뽕,짬뽕,간짜장,후라이드치킨
2,11559,40대,6,비,20.0,25.3,콤비,불고기피자,짬뽕,오리지널피자
3,2460,40대,6,맑음,18.5,27.5,오리지널피자,오리지널피자,짜장면,불고기피자
4,4548,60대 이상,11,맑음,7.5,18.6,짬뽕,짬뽕,짜장면,간짜장
5,2919,20대,11,맑음,9.3,19.8,양념치킨,양념치킨,짜장면,후라이드치킨
6,4955,40대,9,비,20.0,22.3,불고기피자,불고기피자,짜장면,짬뽕
7,5566,50대,2,맑음,-2.3,5.3,후라이드치킨,짬뽕,간짜장,후라이드치킨
8,9478,40대,9,맑음,15.4,23.4,후라이드치킨,불고기피자,오리지널피자,짜장면
9,8180,60대 이상,7,맑음,21.1,27.1,양념치킨,짜장면,양념치킨,콤비


In [333]:
# True Positive, False Negative, False Positive 초기화
TP = 0  # 실제 정답이 추천된 경우
FN = 0  # 실제 정답이 추천되지 않은 경우
FP = 0  # 실제 정답이 아닌 것만 추천된 경우

# 각 사용자에 대한 추천 결과 검사
for _, row in result_df.iterrows():
    actual = row['실제 정답']
    #recommended = [row['추천메뉴 1'], row['추천메뉴 2']]
    recommended = [row['추천메뉴 1'], row['추천메뉴 2'], row['추천메뉴 3']]

    if actual in recommended:
        TP += 1
    else:
        FN += 1
        FP += 1  # 실제 정답이 추천 메뉴 중 어느 것에도 포함되지 않은 경우

# 재현율 및 정밀도 계산
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
precision = TP / (TP + FP) if (TP + FP) > 0 else 0

print("TP:", TP)
print("FN:", FN)
print("FP:", FP)
print("------------------")
print(f"재현율: {recall:.2f}")
print(f"정밀도: {precision:.2f}")

TP: 8
FN: 2
FP: 2
------------------
재현율: 0.80
정밀도: 0.80
